In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../dataSandbox/forPelican/nbaWiggins.csv')

In [3]:
allShots = df[np.invert(np.isnan(df.SHOT_CLOCK))]
cMin,cSec = allShots.loc[:,'GAME_CLOCK'].str.split(':').str.get(0),allShots.loc[:,'GAME_CLOCK'].str.split(':').str.get(1)
allShots.loc[:,'secGameClock'] = cMin.astype('int') * 60 + cSec.astype('int')

/Users/Peter/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Peter/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Dummify variables

In [4]:
pd.get_dummies(allShots[['W','LOCATION','FGM']])
allShots.loc[:,'gameWon'] = pd.get_dummies(allShots['W'])['W']
allShots.loc[:,'homeGame'] = pd.get_dummies(allShots['LOCATION'])['H']
allShots.loc[:,'is3Pointer'] = pd.get_dummies(allShots['PTS_TYPE'])[3]
q = pd.get_dummies(allShots['PERIOD'])[[2,3,4,5]]
allShots = pd.concat([allShots,q],axis=1)

### Clean up

In [5]:
dropThese = ['GAME_ID','MATCHUP','CLOSEST_DEFENDER','CLOSEST_DEFENDER_PLAYER_ID','GAME_CLOCK','SHOT_RESULT','LOCATION','W','PERIOD','PTS_TYPE','PTS']
[ allShots.drop(i, axis=1, inplace=True) for i in dropThese ] 

[None, None, None, None, None, None, None, None, None, None, None]

### Import learning algorithms

In [6]:
from sklearn.linear_model import SGDClassifier, RidgeClassifier, LogisticRegression
from sklearn import cross_validation
from sklearn.preprocessing import scale, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

### Split the data

In [7]:
trainShots = allShots[1::2] #Odd rows
testShots = allShots[::2] #Even rows

### Scale the data
Some algorithms are scale invariant, but others aren't. So scale the data for the appropriate algorithms

### Pluribus machina
Now, build a function to test machine learning algorithms head-to-head.

In [8]:
tempData = pd.DataFrame(scale(allShots,with_std=False), index=allShots.index, columns=allShots.columns)
#tempData = allShots

trainSet = eval('tempData' + '[1::2]').astype(float)
testSet = eval('tempData' + '[::2]').astype(float)

myEst = eval('LogisticRegression()')
predCol = [col for col in allShots.columns if col != 'FGM']
predVar = trainSet['FGM'].astype(float)
predVar.ix[predVar>0]=1
predVar.ix[predVar<=0]=0
myEst.fit(
    X=trainSet[predCol],
    y=predVar
)


testVar = testSet['FGM'].astype(float)
testVar.ix[testVar>0]=1
testVar.ix[testVar<=0]=0

#matches = np.count_nonzero(myEst.predict(testSet[predCol]) == testSet['FGM'])
matches = np.count_nonzero(myEst.predict(testSet[predCol]) == testVar)
#myEst.predict(testSet[predCol])
#myEst.predict(testSet[predCol])
matches

331

In [9]:
#In progress, not working yet...
def myLearning(myData,myMod,trainIdx,testIdx,respVar,scaleIt=False,printMe=False): #Make this a function of train/test set, scale, algorithm
    predCol = [col for col in myData.columns if col != respVar]
    if scaleIt == True:
        myData = pd.DataFrame(scale(myData), index=myData.index, columns=myData.columns)
    #else:
    #    myData = pd.DataFrame(scale(myData, with_std=False), index=myData.index, columns=myData.columns)
    trainSet = eval('myData' + trainIdx).astype(float)
    testSet = eval('myData' + testIdx).astype(float)
    myEst = eval(myMod)
    # Need to binarize data or it will break
    trainResp = trainSet[respVar].astype(float)
    trainResp.ix[trainResp > 0] = 1
    trainResp.ix[trainResp <= 0] = 0
    myEst.fit(
        trainSet[predCol],
        trainResp
    )
    # Need to binarize data or it will break
    testResp = testSet[respVar].astype(float)
    testResp.ix[testResp > 0] = 1
    testResp.ix[testResp <= 0] = 0
    matches = np.count_nonzero(myEst.predict(testSet[predCol]) == testResp)
    testLen = testSet.shape[0]
    if(printMe==True):
        print 'Algorithm used:\n' + myMod + '    scaled=' + str(scaleIt)
        print "Correctly classified:\n" + str(round(float(matches) / testLen * 100,2)) + '%\n\n'
    else:
        return round(float(matches) / testLen * 100,2)

In [23]:
myLearning(myData=allShots,myMod='RandomForestClassifier(n_estimators=100)',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 
myLearning(myData=allShots,myMod='BaggingClassifier()',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 
    
myLearning(myData=allShots,myMod='SGDClassifier(alpha=2)',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 
myLearning(myData=allShots,myMod='RidgeClassifier(alpha=2)',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 

myLearning(myData=allShots,myMod='SVC()',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 
myLearning(myData=allShots,myMod='SVC()',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=True,printMe=True)

myLearning(myData=allShots,myMod='LogisticRegression()',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=True,printMe=True) 
myLearning(myData=allShots,myMod='LogisticRegression()',trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=False,printMe=True) 


Algorithm used:
RandomForestClassifier(n_estimators=100)    scaled=False
Correctly classified:
63.88%


Algorithm used:
BaggingClassifier()    scaled=False
Correctly classified:
60.98%


Algorithm used:
SGDClassifier(alpha=2)    scaled=False
Correctly classified:
55.35%


Algorithm used:
RidgeClassifier(alpha=2)    scaled=False
Correctly classified:
60.62%


Algorithm used:
SVC()    scaled=False
Correctly classified:
54.63%


Algorithm used:
SVC()    scaled=True
Correctly classified:
58.26%


Algorithm used:
LogisticRegression()    scaled=True
Correctly classified:
60.25%


Algorithm used:
LogisticRegression()    scaled=False
Correctly classified:
60.25%




In [24]:
dfPred = pd.DataFrame(columns=['algorithm','accuracy','scaled'])

algs = ['RandomForestClassifier(n_estimators=100)','BaggingClassifier()','SGDClassifier(alpha=2)','RidgeClassifier(alpha=2)','SVC()','LogisticRegression()']
myScale = ['False','False','False','False','True','False']

for i in range(len(algs)):
    # Calculate prediction algorithm for scaled/non-scaled data in each algorithm
    temp1 = myLearning(myData=allShots,myMod=algs[i],trainIdx='[1::2]',testIdx='[::2]',respVar='FGM',scaleIt=myScale[i])
    algName = algs[i]
    algName = algName.split('(')[0]
    dfPred = dfPred.append({'algorithm': algName, 'accuracy': temp1, 'scaled': myScale[i]}, ignore_index = True)
    
dfPred

,algorithm,accuracy,scaled
0,RandomForestClassifier,62.61,False
1,BaggingClassifier,59.17,False
2,SGDClassifier,54.99,False
3,RidgeClassifier,60.62,False
4,SVC,54.63,True
5,LogisticRegression,60.25,False


In [14]:
pwd

u'/Users/Peter/git/pelicanSite/content'

# I THINK THAT THE DATA NEEDS TO BE BINARY

In [15]:
tempData = pd.DataFrame(scale(allShots,with_std=False), index=allShots.index, columns=allShots.columns)
#tempData = allShots

trainSet = eval('tempData' + '[1::2]').astype(float)
testSet = eval('tempData' + '[::2]').astype(float)

myEst = eval('LogisticRegression()')
predCol = [col for col in allShots.columns if col != 'FGM']
predVar = trainSet['FGM'].astype(float)
predVar.ix[predVar>0]=1
predVar.ix[predVar<=0]=0
myEst.fit(
    X=trainSet[predCol],
    y=predVar
)


testVar = testSet['FGM'].astype(float)
testVar.ix[testVar>0]=1
testVar.ix[testVar<=0]=0

#matches = np.count_nonzero(myEst.predict(testSet[predCol]) == testSet['FGM'])
matches = np.count_nonzero(myEst.predict(testSet[predCol]) == testVar)
#myEst.predict(testSet[predCol])
#myEst.predict(testSet[predCol])
matches
#"""

331

In [16]:
ts = trainSet['FGM'].astype(float)
ts.ix[ts>0]=1
ts.ix[ts<0]=0
ts
#trainSet['FGM'].astype(float)

1       1
3       0
5       1
7       1
9       1
11      0
13      0
15      0
17      0
19      1
21      1
23      0
25      0
27      1
29      1
31      1
33      1
35      0
37      1
39      0
41      0
43      0
45      1
47      0
49      1
51      0
53      1
55      0
57      0
59      0
       ..
1059    0
1061    0
1063    1
1065    1
1067    1
1069    0
1071    0
1073    0
1075    0
1077    0
1079    1
1081    1
1083    0
1085    1
1087    0
1089    0
1091    1
1093    0
1095    0
1097    0
1099    1
1101    0
1103    1
1105    0
1107    0
1109    1
1111    1
1113    1
1115    0
1117    1
Name: FGM, dtype: float64

In [17]:
print isinstance(trainSet.loc[1,'FINAL_MARGIN'],float)
print isinstance(allShots.loc[1,'FINAL_MARGIN'],float)

True
True


In [18]:
predCol = [col for col in allShots.columns if col != 'FGM']
predCol

['FINAL_MARGIN',
 'SHOT_NUMBER',
 'SHOT_CLOCK',
 'DRIBBLES',
 'TOUCH_TIME',
 'SHOT_DIST',
 'CLOSE_DEF_DIST',
 'secGameClock',
 'gameWon',
 'homeGame',
 'is3Pointer',
 2,
 3,
 4,
 5]

In [19]:
t = pd.DataFrame(columns=['a','b'])
t = t.append({'a':5,'b':3},ignore_index=True)
t = t.append({'a':6,'b':4},ignore_index=True)

t

,a,b
0,5,3
1,6,4


In [20]:
#pd.DataFrame(scale(allShots,with_std=False), index=allShots.index, columns=allShots.columns)[predCol]

scale(allShots,with_std=False)

array([[ -1.60699364e+01,  -7.06085377e+00,  -9.02089010e-01, ...,
         -2.61580381e-01,  -2.07992734e-01,  -5.44959128e-03],
       [ -1.60699364e+01,  -6.06085377e+00,  -3.30208901e+00, ...,
         -2.61580381e-01,  -2.07992734e-01,  -5.44959128e-03],
       [ -1.60699364e+01,  -5.06085377e+00,  -6.80208901e+00, ...,
         -2.61580381e-01,  -2.07992734e-01,  -5.44959128e-03],
       ..., 
       [  4.93006358e+00,  -5.06085377e+00,   2.89791099e+00, ...,
          7.38419619e-01,  -2.07992734e-01,  -5.44959128e-03],
       [  4.93006358e+00,  -4.06085377e+00,   4.97910990e-01, ...,
          7.38419619e-01,  -2.07992734e-01,  -5.44959128e-03],
       [  4.93006358e+00,  -3.06085377e+00,   2.39791099e+00, ...,
          7.38419619e-01,  -2.07992734e-01,  -5.44959128e-03]])

In [21]:
allShots

,FINAL_MARGIN,SHOT_NUMBER,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,CLOSE_DEF_DIST,FGM,secGameClock,gameWon,homeGame,is3Pointer,2,3,4,5
0,-25,1,11.9,1,3.6,10.5,3.0,0,683,0,1,0,0,0,0,0
1,-25,2,9.5,2,4.2,12.2,2.3,1,334,0,1,0,0,0,0,0
2,-25,3,6.0,0,1.0,24.5,7.0,0,118,0,1,1,0,0,0,0
3,-25,4,14.9,0,0.9,24.9,7.4,0,646,0,1,1,1,0,0,0
4,-25,5,14.8,1,2.2,4.2,3.3,0,432,0,1,0,1,0,0,0
5,-25,6,12.0,1,1.2,15.4,2.3,1,394,0,1,0,1,0,0,0
6,-25,7,24.0,0,0.0,14.4,11.4,1,708,0,1,0,0,1,0,0
7,-25,8,10.2,4,2.7,14.4,3.7,1,669,0,1,0,0,1,0,0
8,-25,9,15.4,0,0.6,4.3,2.3,1,319,0,1,0,0,1,0,0
9,-25,10,12.0,1,1.6,4.6,1.2,1,308,0,1,0,0,0,1,0


In [25]:
pwd

u'/Users/Peter/git/pelicanSite/content'